In [16]:
pip install python-docx python-pptx


   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------- ----------------- 2.1/3.8 MB 9.8 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 9.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ------------------------ --------------- 1.6/2.6 MB 9.3 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 7.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [17]:
import os
import pandas as pd
from PyPDF2 import PdfReader
from pdfminer.high_level import extract_text
from docx import Document
from pptx import Presentation

In [18]:
import os
import pandas as pd
from PyPDF2 import PdfReader
from pdfminer.high_level import extract_text
from docx import Document
from pptx import Presentation

def count_pages_and_check_figures_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    page_count = len(reader.pages)
    
    text = extract_text(pdf_path).lower()
    figure_pages = []
    figure_count = 0
    
    for i, page in enumerate(reader.pages):
        page_text = page.extract_text().lower()
        if any(keyword in page_text for keyword in ['figure', 'fig.']):
            figure_pages.append(i + 1)
            figure_count += 1
    
    has_figures = figure_count > 0
    
    return page_count, has_figures, figure_count, figure_pages

def count_pages_and_check_figures_docx(docx_path):
    doc = Document(docx_path)
    page_count = len(doc.element.xpath('//w:sectPr'))
    figure_count = sum(1 for p in doc.paragraphs if 'figure' in p.text.lower() or 'fig.' in p.text.lower())
    has_figures = figure_count > 0
    figure_pages = []  # Not applicable in the same way as PDFs, but we can list paragraph numbers
    
    if has_figures:
        for i, p in enumerate(doc.paragraphs):
            if 'figure' in p.text.lower() or 'fig.' in p.text.lower():
                figure_pages.append(i + 1)
    
    return page_count, has_figures, figure_count, figure_pages

def count_pages_and_check_figures_pptx(pptx_path):
    presentation = Presentation(pptx_path)
    slide_count = len(presentation.slides)
    figure_count = sum(1 for slide in presentation.slides for shape in slide.shapes if shape.has_text_frame and 'figure' in shape.text.lower() or 'fig.' in shape.text.lower())
    has_figures = figure_count > 0
    figure_pages = []  # Slides with figures
    
    if has_figures:
        for i, slide in enumerate(presentation.slides):
            for shape in slide.shapes:
                if shape.has_text_frame and ('figure' in shape.text.lower() or 'fig.' in shape.text.lower()):
                    figure_pages.append(i + 1)
                    break  # Stop after finding the first figure in the slide
    
    return slide_count, has_figures, figure_count, figure_pages

def process_files_in_subfolders(root_folder):
    results = []

    for subdir, _, files in os.walk(root_folder):
        for file in files:
            file_path = os.path.join(subdir, file)
            if file.endswith('.pdf'):
                page_count, has_figures, figure_count, figure_pages = count_pages_and_check_figures_pdf(file_path)
            elif file.endswith('.docx'):
                page_count, has_figures, figure_count, figure_pages = count_pages_and_check_figures_docx(file_path)
            elif file.endswith('.pptx'):
                page_count, has_figures, figure_count, figure_pages = count_pages_and_check_figures_pptx(file_path)
            else:
                continue  # Skip non-PDF/Word/PowerPoint files
            
            results.append({
                'file_name': file,
                'file_path': file_path,
                'page_count': page_count,
                'has_figures': has_figures,
                'figure_count': figure_count,
                'figure_pages': figure_pages
            })

    return results

def main():
    root_folder = r'C:\Users\(Satang)ChanikarnNik\OneDrive - STelligence Co., Ltd\Documents\GitHub\thaioil\Sample_Doc\Code & Standard'  # Replace with your root folder path
    file_results = process_files_in_subfolders(root_folder)
    
    # Convert the results to a DataFrame
    df = pd.DataFrame(file_results)

    
    # Save the DataFrame to a CSV file
    df.to_csv('document_analysis_results.csv', index=False)

if __name__ == "__main__":
    main()


In [14]:
import os
from PyPDF2 import PdfReader
from pdfminer.high_level import extract_text

def count_pages_and_check_figures(pdf_path):
    reader = PdfReader(pdf_path)
    page_count = len(reader.pages)
    
    text = extract_text(pdf_path).lower()
    has_figures = any(keyword in text for keyword in ['figure', 'fig.'])
    
    return page_count, has_figures

def process_pdfs_in_subfolders(root_folder):
    results = []

    for subdir, _, files in os.walk(root_folder):
        for file in files:
            if file.endswith('.pdf'):
                pdf_path = os.path.join(subdir, file)
                page_count, has_figures = count_pages_and_check_figures(pdf_path)
                results.append({
                    'pdf_path': pdf_path,
                    'page_count': page_count,
                    'has_figures': has_figures
                })

    return results

def main():
    root_folder = r'C:\Users\(Satang)ChanikarnNik\OneDrive - STelligence Co., Ltd\Documents\GitHub\thaioil\Sample_Doc\Code & Standard'  # Replace with your root folder path

    # root_folder = 'your_root_folder_path_here'  # Replace with your root folder path
    pdf_results = process_pdfs_in_subfolders(root_folder)
    
    for result in pdf_results:
        print(f"PDF Path: {result['pdf_path']}")
        print(f"Page Count: {result['page_count']}")
        print(f"Contains Figures: {'Yes' if result['has_figures'] else 'No'}")
        print('-' * 40)

if __name__ == "__main__":
    main()


PDF Path: C:\Users\(Satang)ChanikarnNik\OneDrive - STelligence Co., Ltd\Documents\GitHub\thaioil\Sample_Doc\Code & Standard\API 1004_120.40-095.pdf
Page Count: 30
Contains Figures: Yes
----------------------------------------
PDF Path: C:\Users\(Satang)ChanikarnNik\OneDrive - STelligence Co., Ltd\Documents\GitHub\thaioil\Sample_Doc\Code & Standard\ASME B16.5-2003 Pipe Flanges-Flanged Fittings NPS 12 Through NPS24 MetricInch Standard_120.70-082.pdf
Page Count: 233
Contains Figures: Yes
----------------------------------------
PDF Path: C:\Users\(Satang)ChanikarnNik\OneDrive - STelligence Co., Ltd\Documents\GitHub\thaioil\Sample_Doc\Code & Standard\ASTM Cetane Method For Rating Diesel Fuels 1963_120.80-006.pdf
Page Count: 139
Contains Figures: Yes
----------------------------------------
PDF Path: C:\Users\(Satang)ChanikarnNik\OneDrive - STelligence Co., Ltd\Documents\GitHub\thaioil\Sample_Doc\Code & Standard\IEC 112_120.180-158.pdf
Page Count: 11
Contains Figures: Yes
------------------

In [15]:
import os
from PyPDF2 import PdfReader
from pdfminer.high_level import extract_text

def count_pages_and_check_figures(pdf_path):
    reader = PdfReader(pdf_path)
    page_count = len(reader.pages)
    
    text = extract_text(pdf_path).lower()
    figure_pages = []
    figure_count = 0
    
    for i, page in enumerate(reader.pages):
        page_text = page.extract_text().lower()
        if any(keyword in page_text for keyword in ['figure', 'fig.']):
            figure_pages.append(i + 1)  
            figure_count += 1
    
    has_figures = figure_count > 0
    
    return page_count, has_figures, figure_count, figure_pages

def process_pdfs_in_subfolders(root_folder):
    results = []

    for subdir, _, files in os.walk(root_folder):
        for file in files:
            if file.endswith('.pdf'):
                pdf_path = os.path.join(subdir, file)
                page_count, has_figures, figure_count, figure_pages = count_pages_and_check_figures(pdf_path)
                results.append({
                    'file_name': file,
                    # 'pdf_path': pdf_path,
                    'page_count': page_count,
                    'has_figures': has_figures,
                    'figure_count': figure_count,
                    'figure_pages': figure_pages
                })

    return results

def main():
    root_folder = r'C:\Users\(Satang)ChanikarnNik\OneDrive - STelligence Co., Ltd\Documents\GitHub\thaioil\Sample_Doc\Code & Standard'  # Replace with your root folder path

    # root_folder = 'your_root_folder_path_here'  # Replace with your root folder path
    pdf_results = process_pdfs_in_subfolders(root_folder)
    
    # for result in pdf_results:
    #     print(f"PDF Path: {result['pdf_path']}")
    #     print(f"Page Count: {result['page_count']}")
    #     print(f"Contains Figures: {'Yes' if result['has_figures'] else 'No'}")
    #     print('-' * 40)

    df = pd.DataFrame(pdf_results)
    
    df.to_csv('pdf_analysis_results.csv', index=False)

if __name__ == "__main__":
    main()


In [12]:
def count_pages_and_check_figures(pdf_path):
    reader = PdfReader(pdf_path)
    page_count = len(reader.pages)
    
    text = extract_text(pdf_path).lower()
    figure_pages = []
    figure_count = 0
    
    for i, page in enumerate(reader.pages):
        page_text = page.extract_text().lower()
        if any(keyword in page_text for keyword in ['figure', 'fig.']):
            figure_pages.append(i + 1)  
            figure_count += 1
    
    has_figures = figure_count > 0
    
    return page_count, has_figures, figure_count, figure_pages

def process_pdfs_in_subfolders(root_folder):
    results = []

    for subdir, _, files in os.walk(root_folder):
        for file in files:
            if file.endswith('.pdf'):
                pdf_path = os.path.join(subdir, file)
                page_count, has_figures, figure_count, figure_pages = count_pages_and_check_figures(pdf_path)
                results.append({
                    'file_name': file,
                    'pdf_path': pdf_path,
                    'page_count': page_count,
                    'has_figures': has_figures,
                    'figure_count': figure_count,
                    'figure_pages': figure_pages
                })

    return results

def main():
    root_folder = r'C:\Users\(Satang)ChanikarnNik\OneDrive - STelligence Co., Ltd\Documents\GitHub\thaioil\Sample_Doc\Code & Standard'  # Replace with your root folder path
    pdf_results = process_pdfs_in_subfolders(root_folder)
    
    df = pd.DataFrame(pdf_results)
    
    df.head(5)
    
    df.to_csv('pdf_analysis_results.csv', index=False)

if __name__ == "__main__":
    main()


In [13]:
pdf_results

NameError: name 'pdf_results' is not defined